In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7520243908438134038"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [ ]:
video_path = os.path.abspath("./tmp/video/7520243908438134038.mp4")
audio_path = os.path.abspath("./tmp/video/7520243908438134038.wav")

transcript = """
Știai că România vrea să vândă portul Constanța pentru a reconstrui Ucraina?
 Sună nebunesc? Din păcate e real.
 În cercurile guvernamentale de la București se discută tot mai intens despre vânzarea portului Constanța,
 iar banii obținuți ar urma să fie direcționați către reconstrucția Ucrainei.
 Nu este o glumă, vorbim despre cel mai important port românesc,
 o poartă strategică la Marea Neagră, cu valoare geo-economică și militară uriașă.
 Și se ia în calcul cedarea acestuia, în plină criză globală,
 în favoarea unui plan de solidaritate internațional.
 Întrebarea dură este, solidaritate cu cine și pe spatele cui?
 Ce câștigă România concret din această tranzacție?
 Cine va controla portul după vânzare?
 Cum ajută acest lucru românii care plătesc taxe, facturi și rate tot mai mari?
 În timp ce alte state își apără infrastructura vitală,
 România pare gata să renunțe la ea în numele unor interese geopolitice impuse din afară.
 Nu e doar o problemă economică, e o problemă de suveranitate națională.
 Tu ce crezi? România trebuie să ajute cu orice preț chiar dacă își vinde viitorul?
 Sau ar trebui să punem pe primul loc?
 interesul național, distribuie ca să afle toți ce se pregătește în tăcere.
"""

sentiment_results = {
    'positive': 0.15972021222114563,
    'neutral': 0.0,
    'negative': 0.840279757976532
 }
description = 'Portul Constanța, una dintre cele mai importante infrastructuri strategice ale României, ar putea fi vândut. Guvernul discută ca banii să fie direcționați spre reconstrucția Ucrainei. E o decizie corectă? Sau o  a interesului național #Romania2025 #PortulConstanta #Ucraina #Suveranitate #DistribuieAdevărul'

In [3]:
# video_url, video_path, audio_path = download.tiktok_full(video_id)

INFO - [EnrichmentWorker] - Downloading MP4 for 7520243908438134038 → /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520243908438134038.mp4
INFO - [EnrichmentWorker] - Audio 7520243908438134038 is already downloaded in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7520243908438134038.wav. Skipping


In [4]:
# model_name = "ggml-large-v3.bin" 
# transcript, lang, output = transcribe_audio(audio_path, model_name)
# logging.info(transcript)

INFO - [EnrichmentWorker] - Starting transcriber model ggml-large-v3.bin for audio in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7520243908438134038.wav
INFO - [EnrichmentWorker] - Știai că România vrea să vândă portul Constanța pentru a reconstrui Ucraina?
 Sună nebunesc? Din păcate e real.
 În cercurile guvernamentale de la București se discută tot mai intens despre vânzarea portului Constanța,
 iar banii obținuți ar urma să fie direcționați către reconstrucția Ucrainei.
 Nu este o glumă, vorbim despre cel mai important port românesc,
 o poartă strategică la Marea Neagră, cu valoare geo-economică și militară uriașă.
 Și se ia în calcul cedarea acestuia, în plină criză globală,
 în favoarea unui plan de solidaritate internațional.
 Întrebarea dură este, solidaritate cu cine și pe spatele cui?
 Ce câștigă România concret din această tranzacție?
 Cine va controla portul după vânzare?
 Cum ajută acest lucru românii care plătesc taxe, facturi și rate tot mai 

In [5]:
# sentiment_results = SentimentAnalyzer.get_sentiment(transcript, "ro")
# sentiment_results

INFO - [EnrichmentWorker] - Loading model DGurgurov/xlm-r_romanian_sentiment for language ro
INFO - [EnrichmentWorker] - Successfully loaded model DGurgurov/xlm-r_romanian_sentiment
/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
INFO - [EnrichmentWorker] - SENTIMENT MODEL OUTPUT SequenceClassifierOutput(loss=None, logits=tensor([[ 0.9727, -0.6876]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INFO - [EnrichmentWorker] - SENTIMENT MODEL SCORES [0.84027976 0.15972021]


In [7]:
# description = get_video_description(video_id, db_conn)
# description

'Portul Constanța, una dintre cele mai importante infrastructuri strategice ale României, ar putea fi vândut. Guvernul discută ca banii să fie direcționați spre reconstrucția Ucrainei. E o decizie corectă? Sau o  a interesului național #Romania2025 #PortulConstanta #Ucraina #Suveranitate #DistribuieAdevărul'

In [8]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520243908438134038.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video claims that Romania is considering selling the Port of Constanța to aid in Ukraine's reconstruction. It questions the wisdom of this decision, highlighting the port's strategic, economic, and military importance. The speaker suggests this move might be driven by external geopolitical interests, potentially at the expense of Romania's national sovereignty and its citizens' economic well-being, contrasting it with other nations protecting

In [9]:
analysis_result_json

{'summary': "The video claims that Romania is considering selling the Port of Constanța to aid in Ukraine's reconstruction. It questions the wisdom of this decision, highlighting the port's strategic, economic, and military importance. The speaker suggests this move might be driven by external geopolitical interests, potentially at the expense of Romania's national sovereignty and its citizens' economic well-being, contrasting it with other nations protecting their vital infrastructure.",
 'identified_subjects': [{'subject': 'romania', 'stance': -0.7},
  {'subject': 'ukraine', 'stance': -0.7},
  {'subject': 'port of constanța', 'stance': -0.7},
  {'subject': 'national sovereignty', 'stance': 1.0},
  {'subject': 'geopolitical interests', 'stance': -1.0}],
 'overall_alignment': -0.8}

In [10]:
analysis_result_json

{'summary': "The video claims that Romania is considering selling the Port of Constanța to aid in Ukraine's reconstruction. It questions the wisdom of this decision, highlighting the port's strategic, economic, and military importance. The speaker suggests this move might be driven by external geopolitical interests, potentially at the expense of Romania's national sovereignty and its citizens' economic well-being, contrasting it with other nations protecting their vital infrastructure.",
 'identified_subjects': [{'subject': 'romania', 'stance': -0.7},
  {'subject': 'ukraine', 'stance': -0.7},
  {'subject': 'port of constanța', 'stance': -0.7},
  {'subject': 'national sovereignty', 'stance': 1.0},
  {'subject': 'geopolitical interests', 'stance': -1.0}],
 'overall_alignment': -0.8}